In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from irt import IRTModel
from sklearn import svm
from sklearn.linear_model import SGDRegressor, LinearRegression, BayesianRidge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
# from beta_irt.visualization.plots import newline
# from beta_irt.visualization.plots import plot_parameters
# from irt import beta_irt
from sklearn.decomposition import PCA
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import glob
from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler
from beta_irt.visualization.plots import plot_parameters
from scipy.interpolate import spline

In [ ]:
name = 'mpg'

In [ ]:
# Path
path_data = './data/'
path_uci = './data/UCI - 45/'

# Read csv
data = pd.read_csv(path_uci + name + '.csv')
data = data.dropna()

# Parameters
rd = 42

# Variable selection
X = data.iloc[:, 0].values.reshape(-1,1)
y = data.iloc[:, 1]

# Split data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = rd)

# # Principal component analysis
# pca = PCA(n_components= 1)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

# Standard scale
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [ ]:
names = []
path = './Results_IRT/'+name+'/'
foldernames = glob.glob(path+'*/')
keys = list(map(lambda x: int(x[:-1].split('_')[-1]), foldernames))
keys = sorted(range(len(keys)), key=lambda k: keys[k])
foldernames = list(map(lambda k: foldernames[k], keys))
names = list(map(lambda x: x.split('/')[-2], foldernames))
max_std = int(names[-1].split('_')[-1])

In [ ]:
foldernames

In [ ]:
all_files = []
for i, df in enumerate(foldernames):
    path = df
    filenames = glob.glob(path + "/*.csv")
    
    filenames.sort()

    dfs = []
    for filename in filenames:
        dfs.append(pd.read_csv(filename))
        
    all_files.append(dfs)

In [ ]:
filenames

In [ ]:
all_files[0][-1].shape

In [ ]:
mdls = ['LR', 'Bayes', 'SVR(Linear)', 'SVR(Rbf)', 'KNR', 'DT', 'RF', 'AdaB', 'MLP (100)', 'MLP (50-50)', 'Average', 'Optimal', 'Worst']
abilities = np.zeros((len(mdls), len(foldernames)))
noises = np.zeros((len(foldernames), len(all_files[0][-1])))
errors = np.zeros((len(foldernames), len(all_files[0][-1]), len(mdls) - 3))
responses = np.zeros((len(foldernames), len(all_files[0][-1]), len(mdls)))
params = np.zeros((len(foldernames), len(all_files[0][-1]), 2))
test = np.zeros((len(foldernames), len(all_files[0][-1]), 2))

In [ ]:
for i, data in enumerate(all_files):
    abilities[:, i] = data[1].iloc[:, -1].values
    errors[i] = data[0].values
    responses[i] = data[2].values
    params[i] = data[3].values
    noises[i] = data[-2].values.reshape(1,-1)[0]
    test[i] = data[-1].values

# Noise x MAE

In [ ]:
mae = np.zeros((len(all_files), 10))
for i in range(20):
    mae[i] = np.absolute(errors[i]).mean(axis=0)

In [ ]:
plt.figure(figsize=(18, 9))
for j, model in enumerate(mdls[:10]):
    plt.plot(list(range(20)), mae[:, j], linewidth=0.9, label=model, marker='s',)
plt.xticks(range(0, 20, 2))

ax = plt.subplot(111)
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width*0.65, box.height])
legend_x = 1
legend_y = 0.5

plt.ylabel('Mean Absolute Error (MAE)')
plt.xlabel('Noise level')
plt.legend(loc='center left', bbox_to_anchor=(legend_x, legend_y))
plt.savefig('./Results_IRT/' + name + '/mae_noise.png')

# Noise x Ability

In [ ]:
plt.figure(figsize=(18, 9))
for j, model in enumerate(mdls):
    if j>9:
        plt.plot(list(range(20)), abilities[j, :], linewidth=0.9, label=model, marker='s', linestyle = '--')
    else:
        plt.plot(list(range(20)), abilities[j, :], linewidth=0.9, label=model, marker='o')
plt.xticks(range(0, 20, 2))
plt.xlim([-.1,19.1])
plt.ylim([0.3,0.8])

ax = plt.subplot(111)
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width*0.65, box.height])
legend_x = 1
legend_y = 0.5

plt.ylabel('Ability')
plt.xlabel('Noise level')
plt.legend(loc='center left', bbox_to_anchor=(legend_x, legend_y))
plt.savefig('./Results_IRT/' + name + '/noise_ability.png')

# Histograms Responses

In [ ]:
plts = ['Original', 'Noise 1', 'Noise 2']
noises_i = [0, 9, 19]

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18,3))
scale = 0
for i, noise_i in enumerate(noises_i):
    values = all_files[noise_i][2].values.reshape(-1,1)
    sns.distplot(values, ax= axes[i])
    current_scale = axes[i].get_ylim()[1] - axes[i].get_ylim()[0]
    if current_scale > scale:
        scale = current_scale
        y_lim = axes[i].get_ylim()
    axes[i].set_title(plts[i])
    axes[i].set_xlabel('Response value')
    axes[i].set_ylabel('Frequency')
for ax in axes:
    ax.set_ylim(y_lim)
fig.savefig('./Results_IRT/'+name+'/hists.png')

# IRT Parameters

In [ ]:
for i, n in enumerate(noises_i):
    concat = np.concatenate((test[n][:, 0].reshape(-1,1), test[n][:, 1].reshape(-1,1)), axis = 1)
    f = plot_parameters(concat, delta = params[n][:, 0], a = params[n][:, 1], xlim = [0.85*test[noises_i[-1]][:, 0].min(), 1.15*test[noises_i[-1]][:, 0].max()])
    plt.savefig('./Results_IRT/' + name + '/params_'+ str(i) +'.png')

# Difficulty/ Discrimination

In [ ]:
plot = ["Difficulty", "Discrimination"]
Xsort_i = np.argsort(X_test.reshape(1,-1))
p_mean = params.mean(axis=0)
p_std = params.std(axis=0)
cmap1 = sns.cubehelix_palette(rot=-.5,light=1.5,dark=-.5,as_cmap=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 6))
for i, param in enumerate(plot):
    x = X_test[Xsort_i].reshape(1,-1)[0]
    pmean = p_mean[Xsort_i, i][0]
    pstd = p_std[Xsort_i, i][0]
    
    axes[i].fill_between(x, y1 = pmean - 2*pstd, y2 = pmean + 2*pstd, alpha=0.2)
    axes[i].plot(x, pmean)
    axes[i].plot(x, pmean - 2*pstd, linewidth = 0.5, c='black')
    axes[i].plot(x, pmean + 2*pstd, linewidth = 0.5, c='black')
    axes[i].set_xlabel('X')
    axes[i].set_xlim([x[0], x[-1]])
    axes[i].set_ylabel(param)
plt.savefig('./Results_IRT/' + name + '/params_x.png')
plt.show()

In [ ]:
#Lower noise
scale = 0
fig, axes = plt.subplots(2, 2, figsize=(12,12))

for i, title in enumerate(plot):
    for j,noise in enumerate(noises_i[1:]):
        if i == 0:
            lim = np.array([0,1])
        else:
            lim = np.array([-0.2, 1.2])
        
        axes[i,j].set_xlim(lim)
        axes[i,j].set_ylim(lim)
        axes[i,j].plot([lim[0],lim[1]], [lim[0], lim[1]], linewidth=0.2, c='black')        
        
        sns.set_context('paper')

        axes[i,j].text(x = lim[0], y = lim[0], s='Corr = %.2f' %(spearmanr(a = params[0][:, i], b = params[noise][:, i])[0]))
        axes[i, j].scatter(params[0][:, i], params[noise][:, i], c = noises[noise], cmap = cmap1, edgecolor='k',s=30)

        axes[i,j].set_xlabel("Original data")
        axes[i,j].set_ylabel('Noise ' + str(j+1))
        axes[i,j].set_title(title)
fig.savefig('./Results_IRT/'+name+'/parameters.png')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16,5))
for i, noise in enumerate(noises_i):
    c = np.absolute(noises[i])

    axes[i].scatter(params[noise][:,0], params[noise][:,1], c = c, cmap = cmap1, edgecolor='k',s=30)
    axes[i].set_title(plts[i])
    axes[i].set_xlim([0,1])
    axes[i].set_ylim([-0.2, 1.2])
    axes[i].set_xlabel('Difficulty')
    axes[i].set_ylabel('Discrimination')
fig.savefig('./Results_IRT/'+name+'/dif_disc')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12,10))
for h, param in enumerate(['Diff', 'Discrim']):
    for i, noise in enumerate(noises_i[1:]):
        axes[h, i].scatter(noises[noise], (params[noise][:, h] - params[0][:, h]), edgecolor='k',s=30)
        axes[h, i].set_ylim([-1., 1.])
        axes[h, i].set_xlim([-.2, noises[-1].max() + 0.1])
        axes[h, i].text(x = -.2, y = -1, s='Corr = %.2f' %(spearmanr(a = noises[noise], b = (params[noise][:, h] - params[0][:, h]))[0]))

        axes[h, i].set_xlabel('|Noise|')
        axes[h, i].set_ylabel('Δ '+param+' ('+plts[i+1]+' - Original)')
fig.savefig('./Results_IRT/'+name+'/param_noise')

# Ability

In [ ]:
abilities.shape

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14,6))
mdls = mdls = ['LR', 'Bayes', ' SVR(Lin)', ' SVR(Rbf)', 'KNR', 'DT', 'RF', 'AdaB', 'MLP100', 'MLP50-50', 'Avg', 'Opt', 'Wrs']
lim = [0.4, 0.7]

for i, noise in enumerate(noises_i[1:]):
    for j, mdl in enumerate(mdls):
        axes[i].scatter(abilities[j, 0], abilities[j, noise], s = 50, label = mdl, edgecolor='k')
    axes[i].set_xlim(lim)
    axes[i].set_ylim(lim)
    axes[i].set_xlabel('Original')
    axes[i].set_ylabel('Noise ' + str(i+1))
    axes[i].set_title('Ability')
    axes[i].plot(lim, lim, linewidth=0.2, c='black')
    axes[i].legend()
fig.savefig('./Results_IRT/'+name+'/ability')

# Error

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16,5))
for i, noise in enumerate(noises_i):
    c = np.absolute(noises[noise])
    mean_error = np.absolute(errors[noise]).mean(axis = 1)
    diff = params[noise][:, 0]
    axes[i].scatter(diff,mean_error, c= c, cmap = cmap1, edgecolor='k',s=30)
    axes[i].text(x = 0, y = 0, s='Mean = %.2f Std = %.2f Corr = %.2f' %(mean_error.mean(), mean_error.std(), spearmanr(a = diff, b = mean_error)[0]), fontsize=10)
    axes[i].set_xlabel('Difficulty')
    axes[i].set_ylabel('Mean Error (Instance)')
    axes[i].set_xlim([0,1])
    axes[i].set_ylim([0, 1.05*np.absolute(errors[-1]).mean(axis = 1).max()])
    axes[i].set_title(plts[i])
fig.savefig('./Results_IRT/'+name+'/diff_error')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,5))
for i, noise in enumerate(noises_i[1:]):
    mean_error = np.absolute(errors[noise]).mean(axis = 1)
    axes[i].scatter(noises[noise], mean_error, edgecolor='k',s=30)
    axes[i].text(x = 0, y = 0, s='Corr = %.2f' %(spearmanr(a = noises[noise], b = mean_error)[0]), fontsize=10)
    axes[i].set_xlabel('|Noise|')
    axes[i].set_ylabel('Mean Error (Instance)')
    axes[i].set_xlim([-0.01, 1.5])
    axes[i].set_ylim([-0.1, 1.05*np.absolute(errors[-1]).mean(axis = 1).max()])
    axes[i].set_title(plts[i+1])
fig.savefig('./Results_IRT/'+name+'/noise_error')

In [ ]:
for h, param in enumerate(plot):
    sns.set_context('paper')
    scale = 0
    fig, axes = plt.subplots(1, 3, figsize=(18,2))
    for i, noise in enumerate(noises_i):
        values = params[noise][:, h]
        sns.distplot(values,bins=100,ax=axes[i])
        sns.distplot(values, ax= axes[i])
        current_scale = axes[i].get_ylim()[1] - axes[i].get_ylim()[0]
        if current_scale > scale:
            scale = current_scale
            y_lim = axes[i].get_ylim()
        axes[i].set_title(param + ' (' + plts[i] + ')')
        axes[i].set_ylabel('Frequency')
    for ax in axes:
        ax.set_ylim(y_lim)
    fig.subplots_adjust(hspace=0.3)
    fig.savefig('./Results_IRT/'+name+'/hists_' + param + '.png')

In [ ]:
for i, noise in enumerate(noises_i):
    metrics = pd.DataFrame()
    metrics['Avg. Response'] = responses[noise][:, :-3].mean(axis = 0)
    metrics['Ability'] = abilities[:-3][noise]
    metrics['MAE'] = np.absolute(errors[i]).mean(axis = 0)
    metrics['MSE'] = (errors[]**2).mean(axis = 0)
    print(noise,'\n',metrics.corr(method = 'spearman'),'\n\n')